# Module 3 Coding Assignment: Various Flavors of Gradient Descent
In this notebook, you will leverage what you've learned and implement various flavors of gradient descent including
* Vanilla/Batch Gradient Descent
* Mini-batch Gradient Descent
* Stochastic Gradient Descent

## Instructions:
1. Noise Level Slider: Adjust the noise in the data, which affects the number of outliers around the linear trend.
2. Outlier Slider: Control the number of outliers added to the data.
3. Learning Rate Slider: Change the learning rate for gradient descent optimization.
4. Optimization Method Dropdown: Toggle between Batch Gradient Descent Mini-batch Gradient Descent, and Stochastic Gradient Descent.
5. Iterations Slider: Set the number of iterations for the gradient descent process.

## Note
* When modifying the data (noise, outliers): Do not change the optimization method.
* When modifying the optimization method (learning rate, optimization method, iterations): Keep the data constant (do not adjust the noise/outliers).


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Generate synthetic data for regression with noise and outliers
def generate_data(noise_level, num_outliers):
    np.random.seed(42)
    lenT = 100
    intercept = 4
    slope = 1.5
    x_min, x_max = -2, 2
    range_x = x_max - x_min
    x1 = x_min + range_x * np.random.rand(lenT)
    y = intercept + slope * x1 + noise_level * np.random.randn(lenT)

    # Add outliers
    for i in range(num_outliers):
        y[i] += 10 * np.random.randn(1)

    return x1, y

# Function to calculate loss surface for visualization
def calculate_loss_surface(x1, y, a_range=(-50, 50), b_range=(-50, 50)):
    a = np.linspace(a_range[0], a_range[1], 50)
    b = np.linspace(b_range[0], b_range[1], 50)
    A, B = np.meshgrid(a, b)
    s = np.zeros(A.shape)

    for i in range(len(a)):
        for j in range(len(b)):
            s[j, i] = np.sum((y - (a[i] + b[j] * x1)) ** 2) / len(x1)

    return A, B, s

# Gradient function for the regression problem
def gradF2(params, y, x1):
    intercept, slope = params
    grad_intercept = -2 * np.sum(y - (intercept + slope * x1)) / len(x1)
    grad_slope = -2 * np.sum((y - (intercept + slope * x1)) * x1) / len(x1)
    return np.array([grad_intercept, grad_slope])

# Batch Gradient Descent
def batch_gradient_descent(x1, y, learning_rate, num_iterations):
    intercept, slope = 40, -50
    intercepts, slopes = [intercept], [slope]

    for _ in range(num_iterations):
        grad = gradF2([intercept, slope], y, x1)
        intercept -= learning_rate * grad[0]
        slope -= learning_rate * grad[1]
        intercepts.append(intercept)
        slopes.append(slope)

    return intercepts, slopes

# Mini-Batch Gradient Descent
def mini_batch_gradient_descent(x1, y, learning_rate, num_iterations, batch_size=30):
    intercept, slope = 40, -50
    intercepts, slopes = [intercept], [slope]
    num_batches = int(np.ceil(len(x1) / batch_size))

    for _ in range(num_iterations):
        for batch_idx in range(num_batches):
            start = batch_idx * batch_size
            end = min((batch_idx + 1) * batch_size, len(x1))
            x_batch = x1[start:end]
            y_batch = y[start:end]
            grad = gradF2([intercept, slope], y_batch, x_batch)
            intercept -= learning_rate * grad[0]
            slope -= learning_rate * grad[1]
            intercepts.append(intercept)
            slopes.append(slope)

    return intercepts, slopes

# Stochastic Gradient Descent
def stochastic_gradient_descent(x1, y, learning_rate, num_iterations):
    intercept, slope = 40, -50
    intercepts, slopes = [intercept], [slope]

    for _ in range(num_iterations):
        for i in range(len(x1)):
            grad = gradF2([intercept, slope], y[i:i+1], x1[i:i+1])
            intercept -= learning_rate * grad[0]
            slope -= learning_rate * grad[1]
            intercepts.append(intercept)
            slopes.append(slope)

    return intercepts, slopes

# Interactive dashboard function with contour plot
def update_dashboard(noise_level, num_outliers, learning_rate, optimization_method, num_iterations):
    x1, y = generate_data(noise_level, num_outliers)
    A, B, s = calculate_loss_surface(x1, y)

    if optimization_method == 'Batch Gradient Descent':
        intercepts, slopes = batch_gradient_descent(x1, y, learning_rate, num_iterations)
    elif optimization_method == 'Mini-Batch Gradient Descent':
        intercepts, slopes = mini_batch_gradient_descent(x1, y, learning_rate, num_iterations)
    elif optimization_method == 'Stochastic Gradient Descent':
        intercepts, slopes = stochastic_gradient_descent(x1, y, learning_rate, num_iterations)

    # Create plot
    fig = plt.figure(figsize=(12, 6))

    # Contour plot with optimization path
    ax1 = fig.add_subplot(121)
    ax1.contour(A, B, s, levels=50, cmap='viridis')
    ax1.plot(intercepts, slopes, color='r', marker='o')
    ax1.set_xlabel('Intercept')
    ax1.set_ylabel('Slope')
    ax1.set_title(f'Loss Surface with {optimization_method}')

    # Scatter plot with fitted line
    ax2 = fig.add_subplot(122)
    ax2.scatter(x1, y, label="Data")
    ax2.plot(x1, intercepts[-1] + slopes[-1] * x1, color='r', label="Fitted Line")
    ax2.set_xlabel('x')
    ax2.set_ylabel('y')
    ax2.set_title(f'{optimization_method}')
    ax2.legend()

    plt.tight_layout()
    plt.show()

# Create the interactive widgets
noise_slider = widgets.FloatSlider(value=0.2, min=0, max=2.0, step=0.1, description='Noise Level:')
outlier_slider = widgets.IntSlider(value=10, min=0, max=20, step=1, description='Number of Outliers:')
learning_rate_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Learning Rate:')
optimization_dropdown = widgets.Dropdown(options=['Batch Gradient Descent', 'Mini-Batch Gradient Descent', 'Stochastic Gradient Descent'],
                                         value='Batch Gradient Descent',
                                         description='Optimization Method:')
iterations_slider = widgets.IntSlider(value=50, min=10, max=200, step=10, description='Iterations:')

# Define the interactive output
ui = widgets.VBox([noise_slider, outlier_slider, learning_rate_slider, optimization_dropdown, iterations_slider])
out = widgets.interactive_output(update_dashboard, {
    'noise_level': noise_slider,
    'num_outliers': outlier_slider,
    'learning_rate': learning_rate_slider,
    'optimization_method': optimization_dropdown,
    'num_iterations': iterations_slider
})

# Display the widgets and the output
display(ui, out)


Output()